# Современные методы анализа данных и машинного обучения, БИ

## НИУ ВШЭ, 2023-24 учебный год

# Семинар 21. Natural Language Processing basics

In [ ]:
%%writefile requirements.txt
torch
numpy
pandas
scikit-learn
razdel

In [ ]:
!pip3 install --upgrade -r requirements.txt

## Задача
В рамках сегодняшнего семинара мы будем обучать модели классификации текста. В качестве датасета используем корпус коротких текстов, сформированный на основе русскоязычных сообщений из Twitter. Он содержит 114 991 положительных, 111 923 отрицательных твитов, а также базу неразмеченных твитов объемом 17 639 674 сообщений.


In [ ]:
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv

Перед началом обучения тексты прошли процедуру предварительной обработки:

приведение к нижнему регистру;
замена «ё» на «е»;
замена ссылок на токен «URL»;
замена упоминания пользователя на токен «USER»;
удаление знаков пунктуации.

In [ ]:
import re
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('positive.csv', sep=';', names=n, usecols=['text'])
data_negative = pd.read_csv('negative.csv', sep=';', names=n, usecols=['text'])

sample_size = min(data_positive.shape[0], data_negative.shape[0])
raw_data = np.concatenate((data_positive['text'].values[:sample_size], data_negative['text'].values[:sample_size]), axis=0)

def preprocess_text(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

df_train = pd.DataFrame(columns=['text', 'label'])
df_val = pd.DataFrame(columns=['text', 'label'])
df_test = pd.DataFrame(columns=['text', 'label'])

data = [preprocess_text(t) for t in raw_data]
labels = [1] * sample_size + [0] * sample_size
df_train['text'], df_test['text'], df_train['label'], df_test['label'] = train_test_split(data, labels, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train, test_size=0.2, random_state=1)
df_train

,text,label
85913,раньше все встречались у фонтана в гуме а тепе...,1
42792,ни когда не пойму любовь женщины к женщине хот...,1
85556,два сеанса в кино вот что я люблю парам пам па...,1
36360,я нашел мой наряд для кэти перри USER URL via ...,1
154940,USER ууууу всеее развод обидки и все дела,1
...,...,...
73349,закидываю свой вконтакторостер строчками из пе...,1
109259,гребаный понедельник гребанные 4 пары хочу дом...,0
50057,болячки вроде подживают но губы увеличелись в ...,0
5192,USER я знала что она добрая а за что она тебе,1


Далее проводим процедуру токенизации для наших текстов

In [ ]:
from collections import Counter
from razdel import tokenize


class Vocabulary:
    def __init__(self):
        self.word2index = {
            "<pad>": 0,
            "<unk>": 1
        }
        self.index2word = ["<pad>", "<unk>"]

    def build(self, texts, min_count=7):
        words_counter = Counter(token for tokens in texts for token in tokens)
        for word, count in words_counter.most_common():
            if count >= min_count:
                self.word2index[word] = len(self.word2index)
        self.index2word = [word for word, _ in sorted(self.word2index.items(), key=lambda x: x[1])]

    @property
    def size(self):
        return len(self.index2word)

    def top(self, n=100):
        return self.index2word[1:n+1]

    def get_index(self, word):
        return self.word2index.get(word, 0)

    def get_word(self, index):
        return self.index2word[index]

vocabulary = Vocabulary()
train_texts = df_train["text"].tolist()
train_texts = [[token.text for token in tokenize(text)] for text in train_texts]
vocabulary.build(train_texts)
assert vocabulary.word2index[vocabulary.index2word[10]] == 10
print(vocabulary.size)
print(vocabulary.top(100))

17198
['<unk>', 'USER', 'не', 'я', 'и', 'в', 'на', 'rt', 'а', 'что', 'URL', 'с', 'как', 'у', 'все', 'меня', 'то', 'это', 'так', 'мне', 'd', 'но', 'ты', 'ну', 'по', 'за', 'еще', 'уже', 'вот', 'да', 'же', 'только', 'сегодня', 'о', 'бы', 'нет', 'когда', 'хочу', 'к', 'очень', 'тебя', 'из', 'он', '3', 'день', 'просто', 'мы', 'будет', '2', 'от', 'было', 'если', 'тебе', 'теперь', 'надо', 'даже', 'тоже', 'завтра', 'кто', 'до', 'там', 'его', '1', 'вообще', 'есть', 'для', 'она', 'сейчас', 'спасибо', 'нас', 'буду', 'почему', 'блин', 'могу', 'люблю', 'без', 'знаю', 'вы', 'они', 'тут', 'или', 'раз', 'мой', 'чем', 'ничего', 'со', 'больше', 'всегда', '5', 'хорошо', 'дома', 'про', 'всем', 'можно', 'ее', 'может', 'год', 'потом', 'был', 'спать']


In [ ]:
train_labels = df_train["label"].tolist()
val_labels = df_val["label"].tolist()
test_labels = df_test["label"].tolist()
train_texts = [[token.text for token in tokenize(text)] for text in df_train["text"].tolist()]
val_texts = [[token.text for token in tokenize(text)] for text in df_val["text"].tolist()]
test_texts = [[token.text for token in tokenize(text)] for text in df_test["text"].tolist()]

Далее базовая настройка torch и модели для обучения (пока без архитектуры)

In [ ]:
np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

def get_next_batch(texts, labels, max_seq_len=100, batch_size=128):
    indices = np.arange(len(texts))
    np.random.shuffle(indices)
    batch_begin = 0
    while batch_begin < len(texts):
        batch_indices = indices[batch_begin: batch_begin + batch_size]
        batch = []
        batch_labels = []
        batch_max_len = 0
        for data_ind in batch_indices:
            batch_labels.append(labels[data_ind])
            sample = [vocabulary.get_index(token) for token in texts[data_ind]][:max_seq_len]
            batch_max_len = max(batch_max_len, len(sample))
            sample += [0] * (max_seq_len - len(sample))
            batch.append(sample)
        batch_begin += batch_size
        batch = torch.cuda.LongTensor(batch)[:, :batch_max_len]
        yield batch, torch.cuda.LongTensor(batch_labels)


def train_model(model, texts, labels, val_texts, val_labels, epochs_count=10,
                loss_every_nsteps=1000, lr=0.01, save_path="model.pt", device_name="cuda"):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.CrossEntropyLoss().cuda()
    prev_avg_val_loss = None
    for epoch in range(epochs_count):
        model.train()
        for step, (batch, batch_labels) in enumerate(get_next_batch(texts, labels)):
            logits = model(batch) # Прямой проход
            loss = loss_function(logits, batch_labels) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
        val_total_loss = 0
        val_batch_count = 0
        model.eval()
        for _, (batch, batch_labels) in enumerate(get_next_batch(val_texts, val_labels)):
            logits = model(batch) # Прямой проход
            val_total_loss += loss_function(logits, batch_labels) # Подсчёт ошибки
            val_batch_count += 1
        avg_val_loss = val_total_loss/val_batch_count
        print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
        total_loss = 0
        start_time = time.time()

        if prev_avg_val_loss is not None and avg_val_loss > prev_avg_val_loss:
            model.load_state_dict(torch.load(save_path))
            model.eval()
            break
        prev_avg_val_loss = avg_val_loss
        torch.save(model.state_dict(), save_path)

In [ ]:
from sklearn.metrics import accuracy_score

def test_model(model, texts, labels):
    predicted_labels = []
    true_labels = []
    model.eval()
    for step, (batch, batch_labels) in enumerate(get_next_batch(texts, labels)):
        logits = model(batch) # Прямой проход
        predicted_labels.extend(torch.max(logits.detach().cpu(), 1)[1].numpy())
        true_labels.extend(batch_labels.detach().cpu().numpy())
    print(accuracy_score(true_labels, predicted_labels))

## Сеть прямого распространения

In [ ]:
class FFModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=64, hidden_dim=48):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, embedding_dim)
        self.hidden_layer = nn.Linear(embedding_dim, hidden_dim)
        self.relu_layer = nn.ReLU()
        self.dropout_layer = nn.Dropout(0.2)
        self.out_layer = nn.Linear(hidden_dim, 2)

    def forward(self, inputs):
        projections = self.embeddings_layer.forward(inputs)
        projections = self.dropout_layer(self.relu_layer(self.hidden_layer(projections)))
        pooling = torch.max(projections, 1)[0]
        output = self.out_layer.forward(pooling)
        return output

model = FFModel(vocabulary.size, 64)
train_model(model, train_texts, train_labels, val_texts, val_labels)
test_model(model, test_texts, test_labels)

## Свёрточная сеть
![Conv example](https://image.ibb.co/e6t8ZK/Convolution.gif)

![NLP conv example](https://user-images.githubusercontent.com/6512394/41590312-b1c28fca-73f1-11e8-9123-e26a03853cc7.png)

*From [(Text-Classification-Pytorch)](https://github.com/dongjun-Lee/text-classification-models-tf)*


In [ ]:
class CnnModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=32, filters=((2, 10), (3, 8))):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, embedding_dim)
        self.filters = []
        all_filters_count = 0
        for kernel_size, filters_count in filters:
            all_filters_count += filters_count
            self.filters.append(nn.Conv2d(1, filters_count, (kernel_size, embedding_dim), padding=(1, 0)))
        self.filters = nn.ModuleList(self.filters)
        self.relu_layer = nn.ReLU()
        self.dropout_layer = nn.Dropout(0.2)
        self.out_layer = nn.Linear(all_filters_count, 2)

    def forward(self, inputs):
        projections = self.embeddings_layer.forward(inputs)
        projections = projections.unsqueeze(1)
        # print(projections.size())
        results = []
        for f in self.filters:
            convolved = self.dropout_layer(self.relu_layer(f(projections))).squeeze(3)
            pooling = torch.max(convolved, 2)[0]
            results.append(pooling)
        output = torch.cat(results, 1)
        output = self.out_layer.forward(output)
        return output

model = CnnModel(vocabulary.size, 64)
train_model(model, train_texts, train_labels, val_texts, val_labels)
test_model(model, test_texts, test_labels)

## Рекуррентные сети

![rnn](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)  
*From [(Understanding LSTM Networks)](http://colah.github.io/posts/2015-08-Understanding-LSTMs)*

In [ ]:
class RnnModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=32, filters=((2, 10), (3, 8))):
        super().__init__()

        self.embeddings_layer = nn.Embedding(vocab_size, embedding_dim)
        self.lstm_layer = nn.LSTM(embedding_dim, embedding_dim, batch_first=True)
        self.dropout_layer = nn.Dropout(0.2)
        self.out_layer = nn.Linear(embedding_dim, 2)

    def forward(self, inputs):
        projections = self.embeddings_layer.forward(inputs)
        output, (final_hidden_state, final_cell_state) = self.lstm_layer(projections)
        hidden = self.dropout_layer(final_hidden_state[-1])
        output = self.out_layer.forward(hidden)
        return output

model = RnnModel(vocabulary.size, 64)
train_model(model, train_texts, train_labels, val_texts, val_labels)
test_model(model, test_texts, test_labels)